# Prepare data For Dashboard

In [1]:
import pandas as pd
import numpy as np

In [2]:
ecom_data_df = pd.read_parquet("../data/olist_ecom.parquet")

## --- Data for 'Executive Summary / Overview' and 'Sales & Order Performance' Tab ---

In [ ]:
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]
for col in date_cols:
    ecom_data_df[col] = pd.to_datetime(ecom_data_df[col], errors="coerce")

print("Date columns ensured as datetime.")

Date columns ensured as datetime.


### Monthly Sales Summary

In [4]:
# Make sure 'order_purchase_timestamp' is correctly parsed for time-series operations
ecom_data_df["order_purchase_timestamp"] = pd.to_datetime(
    ecom_data_df["order_purchase_timestamp"]
)

In [6]:
monthly_sales_summary = (
    ecom_data_df.groupby(pd.Grouper(key="order_purchase_timestamp", freq="ME"))
    .agg(
        total_orders=("order_id", "nunique"),
        total_revenue=("price", "sum"),
        total_freight=("freight_value", "sum"),
        avg_order_value=(
            "payment_value",
            "mean",
        ),
    )
    .reset_index()
)

In [7]:
monthly_sales_summary["month_year"] = (
    monthly_sales_summary["order_purchase_timestamp"].dt.to_period("M").astype(str)
)
monthly_sales_summary.drop(columns=["order_purchase_timestamp"], inplace=True)

In [8]:
monthly_sales_summary.to_csv("dashboard_monthly_sales_summary.csv", index=False)
print("dashboard_monthly_sales_summary.csv exported.")

dashboard_monthly_sales_summary.csv exported.


### Daily/Hourly Patterns

In [13]:
# Ensure only 'delivered' orders are counted for typical analysis
delivered_orders = ecom_data_df[ecom_data_df["order_status"] == "delivered"].copy()
delivered_orders["order_purchase_timestamp"] = pd.to_datetime(
    delivered_orders["order_purchase_timestamp"]
)  # Re-ensure datetime

In [15]:
daily_patterns = (
    delivered_orders["order_purchase_timestamp"]
    .dt.day_name()
    .value_counts()
    .reindex(
        ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    )
    .reset_index()
)
daily_patterns.columns = ["day_of_week", "total_orders"]

In [16]:
daily_patterns.to_csv("dashboard_daily_patterns.csv", index=False)
print("dashboard_daily_patterns.csv exported.")

dashboard_daily_patterns.csv exported.


In [17]:
hourly_patterns = (
    delivered_orders["order_purchase_timestamp"]
    .dt.hour.value_counts()
    .sort_index()
    .reset_index()
)
hourly_patterns.columns = ["hour_of_day", "total_orders"]

In [18]:
hourly_patterns.to_csv("dashboard_hourly_patterns.csv", index=False)
print("dashboard_hourly_patterns.csv exported.")

dashboard_hourly_patterns.csv exported.


### Order Status Distribution

In [19]:
order_status_distribution = (
    ecom_data_df["order_status"].value_counts(normalize=True).reset_index()
)
order_status_distribution.columns = ["order_status", "percentage"]

In [ ]:
order_status_distribution.to_csv("dashboard_order_status_distribution.csv", index=False)
print("dashboard_order_status_distribution.csv exported.")

### Delivery Performance Distribution (Overall)

In [20]:
# Use 'delivery_vs_estimate' to classify performance as 'Early', 'On-Time', 'Late' directly from ecom_data_df
# Assuming 'delivery_vs_estimate' column already exists as per your sample output
def classify_delivery_performance(days):
    if pd.isna(days):
        return (
            "Not Delivered/Unknown"  # Handle cases where delivery date might be missing
        )
    if days < 0:
        return "Early"
    elif days == 0:
        return "On-Time"
    else:
        return "Late"

In [21]:
# Create 'delivery_performance_category' by classifying 'delivery_vs_estimate' using a function 'classify_delivery_performance'
ecom_data_df["delivery_performance_category"] = ecom_data_df[
    "delivery_vs_estimate"
].apply(classify_delivery_performance)

# Count the percentage distribution of delivery performance categories and reset index to DataFrame
delivery_performance_counts = (
    ecom_data_df["delivery_performance_category"]
    .value_counts(normalize=True)
    .reset_index()
)

In [22]:
delivery_performance_counts.columns = ["delivery_performance_category", "percentage"]
delivery_performance_counts.to_csv(
    "dashboard_delivery_performance_summary.csv", index=False
)
print("dashboard_delivery_performance_summary.csv exported.")

dashboard_delivery_performance_summary.csv exported.


### Shipping Time Distribution (for histogram) - using the raw column 'total_delivery_time'

In [ ]:
# Drop duplicates if multiple order_items for same order
delivery_time_data = ecom_data_df[["total_delivery_time"]].dropna().drop_duplicates()
delivery_time_data.to_csv("dashboard_delivery_time_distribution.csv", index=False)
print("dashboard_delivery_time_distribution.csv exported.")

dashboard_delivery_time_distribution.csv exported.


### Average Freight Value by Customer State

In [24]:
# Using unique order_id to avoid double counting freight for multiple items in one order
avg_freight_by_state = (
    ecom_data_df.drop_duplicates(subset="order_id")
    .groupby("customer_state")["freight_value"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

In [25]:
avg_freight_by_state.columns = ["customer_state", "avg_freight_value"]
avg_freight_by_state.to_csv("dashboard_avg_freight_by_state.csv", index=False)
print("dashboard_avg_freight_by_state.csv exported.")

dashboard_avg_freight_by_state.csv exported.


## --- Data for 'Customer Insights (RFM Analysis)' Tab ---

### Main RFM Segments Data (already in ecom_data_df, just need unique customer rows)

In [29]:
# Select only relevant RFM columns for each unique customer
customer_rfm_segments = ecom_data_df[
    [
        "customer_unique_id",
        "Recency",
        "Frequency",
        "Monetary",
        "R_score",
        "F_score",
        "M_score",
        "RFM_score_string",
        "RFM_segment",
    ]
].drop_duplicates(subset="customer_unique_id")

In [30]:
customer_rfm_segments.to_csv("dashboard_customer_rfm_segments.csv", index=False)
print("dashboard_customer_rfm_segments.csv exported.")

dashboard_customer_rfm_segments.csv exported.


### RFM Segment Characteristics (Product Category, Payment Type, Customer State)

In [31]:
rfm_product_category_counts = (
    ecom_data_df.groupby(["RFM_segment", "product_category_name"])["order_id"]
    .nunique()
    .reset_index()
)

In [ ]:
rfm_product_category_counts.to_csv(
    "dashboard_rfm_product_category_counts.csv", index=False
)
print("dashboard_rfm_product_category_counts.csv exported.")

dashboard_rfm_product_category_counts.csv exported.


In [33]:
rfm_payment_type_counts = (
    ecom_data_df.groupby(["RFM_segment", "payment_type"])["order_id"]
    .nunique()
    .reset_index()
)

In [34]:
rfm_payment_type_counts.to_csv("dashboard_rfm_payment_type_counts.csv", index=False)
print("dashboard_rfm_payment_type_counts.csv exported.")

dashboard_rfm_payment_type_counts.csv exported.


In [35]:
rfm_customer_state_counts = (
    ecom_data_df.groupby(["RFM_segment", "customer_state"])["order_id"]
    .nunique()
    .reset_index()
)

In [36]:
rfm_customer_state_counts.to_csv("dashboard_rfm_customer_state_counts.csv", index=False)
print("dashboard_rfm_customer_state_counts.csv exported.")

dashboard_rfm_customer_state_counts.csv exported.


### Delivery Performance by RFM Segment

In [37]:
# Aggregate 'delivery_time_days' and 'delivery_vs_estimate' by RFM segment
rfm_delivery_performance_summary = (
    ecom_data_df.drop_duplicates(subset="order_id")
    .groupby("RFM_segment")
    .agg(
        avg_delivery_time=("total_delivery_time", "mean"),
        avg_delivery_vs_estimate=("delivery_vs_estimate", "mean"),
    )
    .reset_index()
)

In [38]:
rfm_delivery_performance_summary.to_csv(
    "dashboard_rfm_delivery_performance_summary.csv", index=False
)
print("dashboard_rfm_delivery_performance_summary.csv exported.")

dashboard_rfm_delivery_performance_summary.csv exported.


## --- Data for 'Seller Performance' Tab ---

In [39]:
# Aggregate Seller Performance Metrics directly from ecom_data_df
seller_summary = (
    ecom_data_df.groupby("seller_id")
    .agg(
        total_sales=("price", "sum"),
        total_freight=("freight_value", "sum"),
        total_orders=("order_id", "nunique"),  # Count unique orders per seller
        avg_delivery_time=("total_delivery_time", "mean"),
        avg_delivery_vs_estimate=("delivery_vs_estimate", "mean"),
    )
    .reset_index()
)

In [40]:
seller_summary["total_revenue"] = (
    seller_summary["total_sales"] + seller_summary["total_freight"]
)
seller_summary.to_csv("dashboard_seller_performance_summary.csv", index=False)
print("dashboard_seller_performance_summary.csv exported.")

dashboard_seller_performance_summary.csv exported.
